In [9]:
import csv
import itertools

all_data=[]
with open("files/combined.tsv") as f:
    comb_reader=csv.reader(f,dialect=csv.excel_tab)
    names=next(comb_reader)
    print(names)
    for x in comb_reader:
        assert(len(x)==6)
        all_data.append(x)
        
        


['id', 'file_platform', 'file_language', 'file_name', 'text', 'labels']


In [26]:
import docx

class TSVDOCX(object):
    
    def __init__(self,path_format="out/out_{counter:05d}.docx",cutoff=950000):
        self.path_format=path_format
        self.file_counter=0
        self.current_file=docx.Document()
        self.current_len=0
        self.cutoff=cutoff
    
    def __del__(self):
        if self.current_len>0:
            self.save_and_make_new()
        
    def save_and_make_new(self):
        fname=self.path_format.format(counter=self.file_counter)
        self.current_file.save(fname)
        self.current_file=docx.Document()
        self.file_counter+=1
        self.current_len=0
        print(fname," ...saved")
        
    def new_example(self,idx,txt):
        if self.current_len+len(txt)+len(str(idx))+3>self.cutoff:
            self.save_and_make_new()
        pgraph=self.current_file.add_paragraph("")
        run=pgraph.add_run(str(idx))
        run.bold=True
        run.underline=True
        pgraph=self.current_file.add_paragraph(txt)
        self.current_len+=len(txt)+len(str(idx))+3
        
        

In [40]:
import bs4
import re

with open("soup.txt","wt") as f:
    for idx, (id,file_platform,file_language,file_name,text,labels) in tqdm.tqdm(enumerate(all_data)):
        if "bretschneider2016en" in file_name:
            soup=bs4.BeautifulSoup(text)
            text=soup.getText()
            text=re.sub("\n +","\n",text).strip()
            text=re.sub("\n{3,}","\n\n",text)
            print(text,file=f,end="\n\n-------------------\n\n")

802385it [00:02, 326246.34it/s] 


In [50]:
import json
import tqdm
import bs4

lang_files={} #lang -> TSVDOCX
for lang in "it,de,fr,tr,id,pt,gr,hi,en".split(","):
    lang_files[lang]=TSVDOCX(path_format="out/out_"+lang+"_{counter:05d}.docx")

skipped={}
with open("examples_meta.jsonl","wt") as meta_f:
    for idx, (id,file_platform,file_language,file_name,text,labels) in tqdm.tqdm(enumerate(all_data)):
        if file_language not in lang_files:
            skipped[file_language]=skipped.get(file_language,0)+1
            continue
        if "wulczyn2017en" in file_name:
            continue
        if not text.strip():
            continue
        if "mandl2019hind" in file_name:
            file_language="hi"
        if "mandl2019ger" in file_name:
            file_language="de"
        if "bretschneider2016en" in file_name:
            soup=bs4.BeautifulSoup(text)
            text=soup.getText()
        text=re.sub("\n +","\n",text).strip()
        text=re.sub("\n{3,}","\n\n",text)
        meta=json.dumps({"idx":idx,"id":id,"file_platform":file_platform,"file_name":file_name,"out_lang":file_language,"orig":(id,file_platform,file_language,file_name,labels)})
        print(meta,file=meta_f)
        lang_files[file_language].new_example(idx,text)
del lang_files
print(skipped)

57374it [00:20, 88804.02it/s]

out/out_en_00000.docx  ...saved


66399it [00:29, 3318.13it/s] 

out/out_en_00001.docx  ...saved


77221it [00:40, 1748.36it/s]

...

out/out_en_00030.docx  ...saved


798780it [05:24, 1652.88it/s]

out/out_en_00031.docx  ...saved
out/out_en_00032.docx  ...saved


802385it [05:30, 2425.18it/s]


out/out_it_00000.docx  ...saved
out/out_de_00001.docx  ...saved
out/out_fr_00000.docx  ...saved
out/out_tr_00000.docx  ...saved
out/out_id_00002.docx  ...saved
out/out_pt_00000.docx  ...saved
out/out_gr_00009.docx  ...saved
out/out_hi_00001.docx  ...saved
out/out_en_00033.docx  ...saved
{'ar': 45276, 'en/fr/it': 856, 'hing': 11843, 'da': 2961, 'mr': 12114}


In [45]:
l1 =set([ text for (id,file_platform,file_language,file_name,text,labels) in all_data if "wulczyn2017en_attack" in file_name])
l2 =set([ text for (id,file_platform,file_language,file_name,text,labels) in all_data if "wulczyn2017en_aggressive" in file_name])
l3 =set([ text for (id,file_platform,file_language,file_name,text,labels) in all_data if "wulczyn2017en_toxic" in file_name])
print(len(l1),len(l2),len(l3))
print(len(l1-l3))

115693 115693 159436
36915


In [48]:
everything=[re.sub("\s+","",text) for (id,file_platform,file_language,file_name,text,labels) in all_data]
print(len(everything))
print(len(set(everything)))

802385
551648


In [33]:
del lang_files